In [52]:
from functools import partial

from astropaul.database import database_connection
import astropaul.html as html
import astropaul.lbt as lbt
import astropaul.phase as ph
import astropaul.targetlistcreator as tlc

%load_ext autoreload
%autoreload 2

import warnings

warnings.simplefilter('always')  # Show all warnings
warnings.showwarning = warnings._showwarnmsg_impl  # Use default formatting with stack trace


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
with database_connection() as conn:
    phase_event_defs = [
        ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.0)),
        ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.05)),
        ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.18)),
        ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.32)),
        ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.45)),
        ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.55)),
        ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.68)),
        ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.82)),
        ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.95)),
    ]

    name = "All Targets"
    creator = tlc.TargetListCreator(name=name, connection=conn, phase_event_defs=phase_event_defs)
    creator.steps = [
        tlc.add_targets,
        # partial(tlc.filter_targets, criteria=lambda df: df["Target Name"] == "TIC 270920077"),
        tlc.ancillary_data_from_tess,
        tlc.add_tess_catalog_associations,
        tlc.add_lists,
        partial(tlc.add_database_table, table_name="ephemerides"),
        partial(tlc.add_database_table, table_name="dssi_observations"),
        partial(tlc.add_database_table, table_name="speckle_detections"),
        partial(tlc.add_database_table, table_name="pepsi_observations"),
        tlc.add_pepsi_evaluations,
        tlc.add_rv_status,
        partial(
            tlc.add_system_configuration,
            table_name="DSSI Observations",
            time_column="Mid JD",
            eclipse_table="SIDE Observations",
        ),
        partial(tlc.add_system_configuration, table_name="PEPSI Observations", time_column="Mid JD"),
        partial(tlc.add_database_table, table_name="zorroalopeke_observations"),
    ]
    tl = creator.calculate(verbose=False)
    html.render_observing_pages(tl, None, {}, "../../Observing Files/All Targets")